# Project Report: Machine Learning
Cassady Jackson and Baylie Schnieder

### Importing packages used for coding

In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, r2_score, f1_score
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import chi2_contingency
from scipy.stats import pearsonr
from scipy.stats import ttest_ind
from scipy.stats import bartlett

### Importing State Data file for analysis

In [136]:
state_data = pd.read_csv('State_Unique_Data.csv')

In [138]:
state_data

,YearEnd,LocationAbbr,State,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,DataValueAlt,...,Stratification1,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,Overall Public School Rank 2021,Higher Ed Quality 2021,School Safety Rank 2021
0,2021,AL,Alabama,NVSS,Cardiovascular Disease,Coronary heart disease mortality among all peo...,"cases per 100,000",Crude Rate,5.6,5.6,...,Age 0-44,1,CVD,CVD08,CRDRATE,AGE,AGE0_44,44,46,17
1,2021,AL,Alabama,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Crude Rate,22.1,22.1,...,Age 0-44,1,CVD,CVD09,CRDRATE,AGE,AGE0_44,44,46,17
2,2021,AL,Alabama,NVSS,Cardiovascular Disease,Coronary heart disease mortality among all peo...,"cases per 100,000",Age-adjusted Rate,85.8,85.8,...,"Black, non-Hispanic",1,CVD,CVD08,AGEADJRATE,RACE,BLK,44,46,17
3,2021,AL,Alabama,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Age-adjusted Rate,58.0,58.0,...,Male,1,CVD,CVD07,AGEADJRATE,SEX,SEXM,44,46,17
4,2021,AL,Alabama,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Crude Rate,273.2,273.2,...,"Black, non-Hispanic",1,CVD,CVD09,CRDRATE,RACE,BLK,44,46,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,2021,WY,Wyoming,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Crude Rate,44.6,44.6,...,Female,56,CVD,CVD07,CRDRATE,SEX,SEXF,19,18,29
2588,2021,WI,Wisconsin,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Crude Rate,18.2,18.2,...,"Multiracial, non-Hispanic",55,CVD,CVD09,CRDRATE,RACE,MRC,8,5,24
2589,2021,WY,Wyoming,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Age-adjusted Rate,159.4,159.4,...,Overall,56,CVD,CVD09,AGEADJRATE,OVERALL,OVR,19,18,29
2590,2021,WI,Wisconsin,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Age-adjusted Rate,34.6,34.6,...,"White, non-Hispanic",55,CVD,CVD07,AGEADJRATE,RACE,WHT,8,5,24


### Filtering data to be used for linear regression

In [141]:
states_data = state_data.copy()
states_data.drop(['StratificationID1','StratificationCategoryID1','DataValueTypeID','QuestionID', 'TopicID', 'StratificationCategory1', 'DataValueType', 'DataValueUnit', 'Question','DataSource', 'LocationID', 'Stratification1', 'LocationAbbr', 'State', 'Topic', 'YearEnd', 'DataValueAlt', 'LowConfidenceLimit', 'HighConfidenceLimit' ], axis=1, inplace=True)
display(states_data)

,DataValue,Overall Public School Rank 2021,Higher Ed Quality 2021,School Safety Rank 2021
0,5.6,44,46,17
1,22.1,44,46,17
2,85.8,44,46,17
3,58.0,44,46,17
4,273.2,44,46,17
...,...,...,...,...
2587,44.6,19,18,29
2588,18.2,8,5,24
2589,159.4,19,18,29
2590,34.6,8,5,24


### Grouping by Top 10 State Rankings and Bottom 10 State Rankings
Also saving each filtered dataframe to its own file to be used in further analysis as well. 

In [144]:
def top_ten(state):
    if state <= 10:
        return 'Top 10'
    elif 10 < state <= 41:
        return 'Middle'
    else:
        return 'Bottom 10'

state_data['Ranking Level'] = state_data['Overall Public School Rank 2021'].apply(top_ten)

display(state_data)

,YearEnd,LocationAbbr,State,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,DataValueAlt,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,Overall Public School Rank 2021,Higher Ed Quality 2021,School Safety Rank 2021,Ranking Level
0,2021,AL,Alabama,NVSS,Cardiovascular Disease,Coronary heart disease mortality among all peo...,"cases per 100,000",Crude Rate,5.6,5.6,...,1,CVD,CVD08,CRDRATE,AGE,AGE0_44,44,46,17,Bottom 10
1,2021,AL,Alabama,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Crude Rate,22.1,22.1,...,1,CVD,CVD09,CRDRATE,AGE,AGE0_44,44,46,17,Bottom 10
2,2021,AL,Alabama,NVSS,Cardiovascular Disease,Coronary heart disease mortality among all peo...,"cases per 100,000",Age-adjusted Rate,85.8,85.8,...,1,CVD,CVD08,AGEADJRATE,RACE,BLK,44,46,17,Bottom 10
3,2021,AL,Alabama,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Age-adjusted Rate,58.0,58.0,...,1,CVD,CVD07,AGEADJRATE,SEX,SEXM,44,46,17,Bottom 10
4,2021,AL,Alabama,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Crude Rate,273.2,273.2,...,1,CVD,CVD09,CRDRATE,RACE,BLK,44,46,17,Bottom 10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,2021,WY,Wyoming,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Crude Rate,44.6,44.6,...,56,CVD,CVD07,CRDRATE,SEX,SEXF,19,18,29,Middle
2588,2021,WI,Wisconsin,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Crude Rate,18.2,18.2,...,55,CVD,CVD09,CRDRATE,RACE,MRC,8,5,24,Top 10
2589,2021,WY,Wyoming,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Age-adjusted Rate,159.4,159.4,...,56,CVD,CVD09,AGEADJRATE,OVERALL,OVR,19,18,29,Middle
2590,2021,WI,Wisconsin,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Age-adjusted Rate,34.6,34.6,...,55,CVD,CVD07,AGEADJRATE,RACE,WHT,8,5,24,Top 10


In [146]:
top_ten = state_data.copy()
bottom_ten = state_data.copy()

In [148]:
top_ten = top_ten.drop(top_ten[top_ten['Ranking Level'] != 'Top 10'].index)
display(top_ten)
top_ten.to_csv('TopTenStates.csv', header=True, index=False, encoding="utf-8")

,YearEnd,LocationAbbr,State,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,DataValueAlt,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,Overall Public School Rank 2021,Higher Ed Quality 2021,School Safety Rank 2021,Ranking Level
53,2021,CT,Connecticut,NVSS,Cardiovascular Disease,Coronary heart disease mortality among all peo...,"cases per 100,000",Crude Rate,2.4,2.4,...,9,CVD,CVD08,CRDRATE,AGE,AGE0_44,2,2,19,Top 10
54,2021,CT,Connecticut,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Age-adjusted Rate,29.5,29.5,...,9,CVD,CVD07,AGEADJRATE,OVERALL,OVR,2,2,19,Top 10
55,2021,DE,Delaware,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Crude Rate,55.3,55.3,...,10,CVD,CVD09,CRDRATE,RACE,ASN,9,15,2,Top 10
69,2021,DE,Delaware,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Crude Rate,75.3,75.3,...,10,CVD,CVD07,CRDRATE,OVERALL,OVR,9,15,2,Top 10
72,2021,CT,Connecticut,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Crude Rate,14.0,14.0,...,9,CVD,CVD07,CRDRATE,RACE,HIS,2,2,19,Top 10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2584,2021,WI,Wisconsin,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Crude Rate,20.2,20.2,...,55,CVD,CVD07,CRDRATE,RACE,ASN,8,5,24,Top 10
2585,2021,WI,Wisconsin,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Age-adjusted Rate,35.8,35.8,...,55,CVD,CVD07,AGEADJRATE,SEX,SEXM,8,5,24,Top 10
2586,2021,WI,Wisconsin,NVSS,Cardiovascular Disease,Coronary heart disease mortality among all peo...,"cases per 100,000",Crude Rate,118.2,118.2,...,55,CVD,CVD08,CRDRATE,OVERALL,OVR,8,5,24,Top 10
2588,2021,WI,Wisconsin,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Crude Rate,18.2,18.2,...,55,CVD,CVD09,CRDRATE,RACE,MRC,8,5,24,Top 10


In [150]:
bottom_ten = bottom_ten.drop(bottom_ten[bottom_ten['Ranking Level'] != 'Bottom 10'].index)
display(bottom_ten)
bottom_ten.to_csv('BottomTen.csv', header=True, index=False, encoding="utf-8")

,YearEnd,LocationAbbr,State,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,DataValueAlt,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,Overall Public School Rank 2021,Higher Ed Quality 2021,School Safety Rank 2021,Ranking Level
0,2021,AL,Alabama,NVSS,Cardiovascular Disease,Coronary heart disease mortality among all peo...,"cases per 100,000",Crude Rate,5.6,5.6,...,1,CVD,CVD08,CRDRATE,AGE,AGE0_44,44,46,17,Bottom 10
1,2021,AL,Alabama,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Crude Rate,22.1,22.1,...,1,CVD,CVD09,CRDRATE,AGE,AGE0_44,44,46,17,Bottom 10
2,2021,AL,Alabama,NVSS,Cardiovascular Disease,Coronary heart disease mortality among all peo...,"cases per 100,000",Age-adjusted Rate,85.8,85.8,...,1,CVD,CVD08,AGEADJRATE,RACE,BLK,44,46,17,Bottom 10
3,2021,AL,Alabama,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Age-adjusted Rate,58.0,58.0,...,1,CVD,CVD07,AGEADJRATE,SEX,SEXM,44,46,17,Bottom 10
4,2021,AL,Alabama,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Crude Rate,273.2,273.2,...,1,CVD,CVD09,CRDRATE,RACE,BLK,44,46,17,Bottom 10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2114,2021,OK,Oklahoma,NVSS,Cardiovascular Disease,Coronary heart disease mortality among all peo...,"cases per 100,000",Crude Rate,169.3,169.3,...,40,CVD,CVD08,CRDRATE,RACE,WHT,47,48,20,Bottom 10
2116,2021,SC,South Carolina,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Crude Rate,59.1,59.1,...,45,CVD,CVD07,CRDRATE,OVERALL,OVR,43,42,41,Bottom 10
2117,2021,SC,South Carolina,NVSS,Cardiovascular Disease,Coronary heart disease mortality among all peo...,"cases per 100,000",Age-adjusted Rate,88.0,88.0,...,45,CVD,CVD08,AGEADJRATE,RACE,WHT,43,42,41,Bottom 10
2121,2021,SC,South Carolina,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Crude Rate,61.6,61.6,...,45,CVD,CVD07,CRDRATE,SEX,SEXF,43,42,41,Bottom 10


### Grouping by higher education quality for top 25 states and bottom 25.
Also saving to files to be able to use in other files for visualizations and descriptive statistics

In [155]:
def half(state):
    if state <= 25:
        return 'Top 25'
    else:
        return 'Bottom 25'

state_data['Higher Ed Ranking Level'] = state_data['Higher Ed Quality 2021'].apply(half)

display(state_data)

,YearEnd,LocationAbbr,State,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,DataValueAlt,...,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,Overall Public School Rank 2021,Higher Ed Quality 2021,School Safety Rank 2021,Ranking Level,Higher Ed Ranking Level
0,2021,AL,Alabama,NVSS,Cardiovascular Disease,Coronary heart disease mortality among all peo...,"cases per 100,000",Crude Rate,5.6,5.6,...,CVD,CVD08,CRDRATE,AGE,AGE0_44,44,46,17,Bottom 10,Bottom 25
1,2021,AL,Alabama,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Crude Rate,22.1,22.1,...,CVD,CVD09,CRDRATE,AGE,AGE0_44,44,46,17,Bottom 10,Bottom 25
2,2021,AL,Alabama,NVSS,Cardiovascular Disease,Coronary heart disease mortality among all peo...,"cases per 100,000",Age-adjusted Rate,85.8,85.8,...,CVD,CVD08,AGEADJRATE,RACE,BLK,44,46,17,Bottom 10,Bottom 25
3,2021,AL,Alabama,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Age-adjusted Rate,58.0,58.0,...,CVD,CVD07,AGEADJRATE,SEX,SEXM,44,46,17,Bottom 10,Bottom 25
4,2021,AL,Alabama,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Crude Rate,273.2,273.2,...,CVD,CVD09,CRDRATE,RACE,BLK,44,46,17,Bottom 10,Bottom 25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,2021,WY,Wyoming,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Crude Rate,44.6,44.6,...,CVD,CVD07,CRDRATE,SEX,SEXF,19,18,29,Middle,Top 25
2588,2021,WI,Wisconsin,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Crude Rate,18.2,18.2,...,CVD,CVD09,CRDRATE,RACE,MRC,8,5,24,Top 10,Top 25
2589,2021,WY,Wyoming,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Age-adjusted Rate,159.4,159.4,...,CVD,CVD09,AGEADJRATE,OVERALL,OVR,19,18,29,Middle,Top 25
2590,2021,WI,Wisconsin,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Age-adjusted Rate,34.6,34.6,...,CVD,CVD07,AGEADJRATE,RACE,WHT,8,5,24,Top 10,Top 25


In [163]:
top_25 = state_data.copy()

In [165]:
top_25 = top_25.drop(top_25[top_25['Higher Ed Ranking Level'] != 'Top 25'].index)
top_25.to_csv('Top25.csv', header=True, index=False, encoding="utf-8")

In [159]:
bottom_25 = state_data.copy()

In [161]:
bottom_25 = bottom_25.drop(bottom_25[bottom_25['Higher Ed Ranking Level'] != 'Bottom 25'].index)
bottom_25.to_csv('Bottom25.csv', header=True, index=False, encoding="utf-8")

### Grouping by crude rate versus age adjusted rate of cardiovascular disease
Also saving files to be able to use in further analysis and visualizations

In [178]:
ageadj = state_data.copy()
cruderate = state_data.copy()

In [180]:
ageadj = ageadj.drop(ageadj[ageadj['DataValueType'] != 'Age-adjusted Rate'].index)
ageadj.to_csv('AgeAdjustedRate.csv', header=True, index=False, encoding="utf-8")

In [182]:
cruderate = cruderate.drop(cruderate[cruderate['DataValueType'] != 'Crude Rate'].index)
cruderate.to_csv('CrudeRate.csv', header=True, index=False, encoding="utf-8")

### Hypothesis Testing
Compairng across the top 10 states and bottom 10 states to see if there is a statistically significant difference

In [194]:
# separate data values for top 10 and bottom 10 states. 
top = state_data[state_data["Ranking Level"] == "Top 10"]["DataValue"]
bottom = state_data[state_data["Ranking Level"] == "Bottom 10"]["DataValue"]

# perform Welch's t-test (default for unequal variances)
t_stat, p_value = ttest_ind(top, bottom, equal_var=False)

# display the results
print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value: {p_value:.4f}")

# interpretation
alpha = 0.05  # Significance level
if p_value < alpha:
    print("Reject H0: The instances of cardiovascular disease are statistically significant between the top and bottom 10 states.")
else:
    print("Fail to reject H0: No significant difference in the instances of cardiovascular disease between the top and bottom 10 states.")

T-Statistic: -2.9385
P-Value: 0.0034
Reject H0: The instances of cardiovascular disease are statistically significant between the top and bottom 10 states.


#### Shapiro testing the data for a normal distribution

In [88]:
# extract payments
datavalues = state_data["DataValue"]

# Shapiro-Wilk test for normality
stat, p_value = stats.shapiro(datavalues)

# display the results
print(f"Shapiro-Wilk Test Statistic: {stat}")
print(f"P-Value: {p_value}")

# interpretation
alpha = 0.05  # Significance level
if p_value > alpha:
    print("The data follows a normal distribution (fail to reject H0).")
else:
    print("The data does not follow a normal distribution (reject H0).")

Shapiro-Wilk Test Statistic: 0.6012805510147978
P-Value: 7.192250613446603e-61
The data does not follow a normal distribution (reject H0).


### Hypothesis Testing 2
Comparing across the top 25 and bottom 25 states for higher education quality to see if there is a statistically significant difference.

In [186]:
# separate data values for top 25 and bottom 25 states. 
top = state_data[state_data["Higher Ed Ranking Level"] == "Top 25"]["DataValue"]
bottom = state_data[state_data["Higher Ed Ranking Level"] == "Bottom 25"]["DataValue"]

# perform Welch's t-test (default for unequal variances)
t_stat, p_value = ttest_ind(top, bottom, equal_var=False)

# display the results
print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value: {p_value:.4f}")

# interpretation
alpha = 0.05  # Significance level
if p_value < alpha:
    print("Reject H0: The instances of cardiovascular disease are statistically significant between the top and bottom 25 states.")
else:
    print("Fail to reject H0: No significant difference in the instances of cardiovascular disease between the top and bottom 25 states.")

T-Statistic: -1.7304
P-Value: 0.0837
Fail to reject H0: No significant difference in the instances of cardiovascular disease between the top and bottom 25 states.


### Hypothesis Testing 3
Comparing age adjusted rate and crude rate to see if there is a statistically significant difference

In [191]:
# separate data values for age adjusted and crude rates 
age_adj = ageadj["DataValue"]
crude_rate = cruderate["DataValue"]

# perform Welch's t-test (default for unequal variances)
t_stat, p_value = ttest_ind(age_adj, crude_rate, equal_var=False)

# display the results
print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value: {p_value:.4f}")

# interpretation
alpha = 0.05  # Significance level
if p_value < alpha:
    print("Reject H0: The instances of cardiovascular disease are statistically significant between the age adjusted and crude rates.")
else:
    print("Fail to reject H0: No significant difference in the instances of cardiovascular disease between the the age adjusted and crude rates.")

T-Statistic: -8.2585
P-Value: 0.0000
Reject H0: The instances of cardiovascular disease are statistically significant between the age adjusted and crude rates.


### Linear Regression
Trying to predict rate of cardiovascular disease using education rankings

In [197]:
states_data.head()
X = states_data.drop('DataValue', axis=1)
y = states_data['DataValue'].values
X_train, X_test, y_train, y_test = train_test_split(X ,y, test_size = 0.2, random_state = 42)
Lin_Regression = LinearRegression().fit(X_train, y_train)
y_pre = Lin_Regression.predict(X_test)
Lin_Regression.score(X_test, y_test)

0.004217637019521825

### Logistic Regression
Trying to predict crude versus age adjusted rate of cardiovascular disease using education ranking variables

In [226]:
states_data.head()
X = states_data.drop('DataValue', axis=1)
y = state_data['DataValueType'].values
X_train, X_test, y_train, y_test = train_test_split(X ,y, test_size = 0.2, random_state = 42)
log_reg = LogisticRegression(max_iter = 5000)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
print(f'Logistic Regression Score: {log_reg.score(X_test, y_test)}')

Logistic Regression Score: 0.5626204238921002


### Pearson Correlation

Used to understand the strength and direction of the linear relationship between two variables.

In [93]:
# calculate Pearson correlation between 'Bottom 10' Ranking and DataValueUnit (instances of chronic illness) 
pearson_corr = bottom_ten["Overall Public School Rank 2021"].corr(bottom_ten["DataValue"], method="pearson")

# display the result
print(f"Pearson Correlation: {pearson_corr:.4f}")

# interpretation
if abs(pearson_corr) < 0.3:
    print("Interpretation: No or weak linear correlation.")
elif 0.3 <= abs(pearson_corr) < 0.7:
    print("Interpretation: Moderate linear correlation.")
else:
    print("Interpretation: Strong linear correlation.")

Pearson Correlation: -0.0430
Interpretation: No or weak linear correlation.


Used to test the hypothesis found in the Pearson correlation and understand if it is significant or could have occured by chance

In [95]:
# perform the Pearson correlation test
corr_stat, p_value = pearsonr(bottom_ten["Overall Public School Rank 2021"], bottom_ten["DataValue"])

# display the correlation statistic and p-value
print(f"Pearson Correlation Coefficient: {corr_stat:.4f}")
print(f"P-value: {p_value:.4f}")

# interpretation based on p-value
alpha = 0.05  # significance level

if p_value > alpha:
    print("Fail to reject H0: No significant correlation (correlation = 0).")
else:
    print("Reject H0: Significant correlation exists (correlation != 0).")

Pearson Correlation Coefficient: -0.0430
P-value: 0.3159
Fail to reject H0: No significant correlation (correlation = 0).


In [96]:
# calculate Pearson correlation between 'Bottom 10' Ranking and DataValueUnit (instances of chronic illness) 
top_pearson_corr = top_ten["Overall Public School Rank 2021"].corr(top_ten["DataValue"], method="pearson")

# display the result
print(f"Pearson Correlation: {top_pearson_corr:.4f}")

# interpretation
if abs(top_pearson_corr) < 0.3:
    print("Interpretation: No or weak linear correlation.")
elif 0.3 <= abs(top_pearson_corr) < 0.7:
    print("Interpretation: Moderate linear correlation.")
else:
    print("Interpretation: Strong linear correlation.")

Pearson Correlation: 0.0513
Interpretation: No or weak linear correlation.


In [97]:
# perform the Pearson correlation test
corr_stat, p_value = pearsonr(top_ten["Overall Public School Rank 2021"], top_ten["DataValue"])

# display the correlation statistic and p-value
print(f"Pearson Correlation Coefficient: {corr_stat:.4f}")
print(f"P-value: {p_value:.4f}")

# interpretation based on p-value
alpha = 0.05  # significance level

if p_value > alpha:
    print("Fail to reject H0: No significant correlation (correlation = 0).")
else:
    print("Reject H0: Significant correlation exists (correlation != 0).")

Pearson Correlation Coefficient: 0.0513
P-value: 0.2538
Fail to reject H0: No significant correlation (correlation = 0).


In [98]:
state_data

,YearEnd,LocationAbbr,State,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,DataValueAlt,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,Overall Public School Rank 2021,Higher Ed Quality 2021,School Safety Rank 2021,Ranking Level
0,2021,AL,Alabama,NVSS,Cardiovascular Disease,Coronary heart disease mortality among all peo...,"cases per 100,000",Crude Rate,5.6,5.6,...,1,CVD,CVD08,CRDRATE,AGE,AGE0_44,44,46,17,Bottom 10
1,2021,AL,Alabama,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Crude Rate,22.1,22.1,...,1,CVD,CVD09,CRDRATE,AGE,AGE0_44,44,46,17,Bottom 10
2,2021,AL,Alabama,NVSS,Cardiovascular Disease,Coronary heart disease mortality among all peo...,"cases per 100,000",Age-adjusted Rate,85.8,85.8,...,1,CVD,CVD08,AGEADJRATE,RACE,BLK,44,46,17,Bottom 10
3,2021,AL,Alabama,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Age-adjusted Rate,58.0,58.0,...,1,CVD,CVD07,AGEADJRATE,SEX,SEXM,44,46,17,Bottom 10
4,2021,AL,Alabama,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Crude Rate,273.2,273.2,...,1,CVD,CVD09,CRDRATE,RACE,BLK,44,46,17,Bottom 10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587,2021,WY,Wyoming,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Crude Rate,44.6,44.6,...,56,CVD,CVD07,CRDRATE,SEX,SEXF,19,18,29,Middle
2588,2021,WI,Wisconsin,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Crude Rate,18.2,18.2,...,55,CVD,CVD09,CRDRATE,RACE,MRC,8,5,24,Top 10
2589,2021,WY,Wyoming,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Age-adjusted Rate,159.4,159.4,...,56,CVD,CVD09,AGEADJRATE,OVERALL,OVR,19,18,29,Middle
2590,2021,WI,Wisconsin,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Age-adjusted Rate,34.6,34.6,...,55,CVD,CVD07,AGEADJRATE,RACE,WHT,8,5,24,Top 10


In [99]:
states_data

,DataValue,Overall Public School Rank 2021,Higher Ed Quality 2021,School Safety Rank 2021
0,5.6,44,46,17
1,22.1,44,46,17
2,85.8,44,46,17
3,58.0,44,46,17
4,273.2,44,46,17
...,...,...,...,...
2587,44.6,19,18,29
2588,18.2,8,5,24
2589,159.4,19,18,29
2590,34.6,8,5,24


In [100]:
XKNN = states_data.drop('Overall Public School Rank 2021', axis=1)
yKNN = states_data['Overall Public School Rank 2021'].values
XKNN_train, XKNN_test, yKNN_train, yKNN_test = train_test_split(XKNN ,yKNN, test_size = 0.2, random_state = 42)
knn = KNeighborsClassifier(n_neighbors =7)
knn.fit(XKNN_train, yKNN_train)
yKNN_pred = knn.predict(XKNN_test)
print(confusion_matrix(yKNN_test, yKNN_pred))

[[ 4  0  0 ...  0  0  0]
 [ 0  2  3 ...  0  0  0]
 [ 0  9  7 ...  0  0  0]
 ...
 [ 0  0  0 ... 10  0  0]
 [ 0  0  0 ...  0  1  2]
 [ 0  0  0 ...  0  4  7]]


In [101]:
print(classification_report(yKNN_test, yKNN_pred))

              precision    recall  f1-score   support

           1       0.44      0.44      0.44         9
           2       0.15      0.33      0.21         6
           3       0.58      0.41      0.48        17
           4       0.25      0.38      0.30         8
           5       0.07      0.25      0.11         4
           6       0.33      0.11      0.17         9
           7       0.50      0.42      0.45        12
           8       0.33      0.56      0.42         9
           9       0.57      0.44      0.50         9
          10       0.25      0.50      0.33         4
          11       0.43      0.25      0.32        12
          12       0.80      0.67      0.73        12
          13       0.38      0.50      0.43         6
          14       0.00      0.00      0.00         1
          15       0.60      0.69      0.64        13
          16       0.38      0.50      0.43        10
          17       0.64      0.64      0.64        11
          18       0.62    

In [102]:
f1_score(yKNN_test, yKNN_pred, average= 'weighted')

0.45205510781692027